# 10 - Systems of Equations
- 10.1 Algebra and geometry of equations
- 10.2 From systems to matrices
- 10.3 Row reduction
- 10.4 Gaussian elimination
- 10.5 Row-reduced echelon form
- 10.6 Gauss-Jordan elimination
- 10.7 Possibilities for solutions
- 10.8 Matrix spaces, row reduction
- 10.9 Exercises
- 10.10 Answers
- 10.11 Coding challenges
- 10.12 Code solutions

Notes, code snippets, and the end of chapter exercises from the book _Linear Algebra: Theory, Intuition, Code_ by Mike X Cohen. 

Find more information about the book on [github](https://github.com/mikexcohen/LinAlgBook) and [amazon](https://www.amazon.com/Linear-Algebra-Theory-Intuition-Code/dp/9083136604).

In [1]:
import numpy as np
import sympy as sym

## 10.1 Algebra and geometry of equations
Every solution to the system of equations $Ax = b$ has one of the following outcomes:

- If $A$ is nonsingular, then there is exactly one (unique) solution
    - $b \in \text{span}(A)$ and linearly independent
    - Geometric interpretation: Lines intersect at the point $x$ with coordinates equal to the solution
- If $A$ is singular and $b \in \text{span}(A)$, then there are an infinite number of solutions
    - Geometric interpretation: Colinear lines
- If $A$ is singular and $b \notin \text{span}(A)$, then there is no solution
    - Geometric interpretation: Parallel lines

Notes
- Existence of unique solution depends only on $A$

### Singularity
A nonsingular matrix $A$ has the following properties:
1. There exists $A^{-1}$ such that $A^{-1} A = A A^{-1} = I$
2. $\text{det}(A) \neq 0$
3. $\text{rank}(A) = n$ where $A$ is $n \times n$ matrix
4. Null space of matrix is empty $N(A) = \emptyset$ e.g. $Az \neq 0$ for any vector $z \neq 0$

## 10.2 From systems to matrices
Terminology used in system of equations $Ax = b$

- Variables: $x$
    - Unknowns that solve the system
- Coefficients: $A$
    - Matrix of coefficients, one equation per row
- Constants: $b$
    - Right hand side, do not participate in multiplication $Ax$

## 10.3 Row reduction

### Row Echelon Form aka REF
Row echelon form of a matrix $\text{REF}(A)$ corresponds to an upper triangular matrix

Example
$$
\text{REF}(A) = 
\begin{bmatrix}
a & b & c \\
0 & d & e \\
0 & 0 & f \\
\end{bmatrix}
$$

Notes
- Leftmost nonzero elements in each row are known as pivots
- REF form of a matrix is not unique

### Elementary Row Operations (ERO)
Use elementary row operations (listed below) to transform a matrix to row echelon form.

1. Swap rows
2. Scale a row by a nonzero constant
3. Add to a row the scalar multiple of another row

Notes
- Matrix rank is equal to...
    - ... number of nonzero rows in row echelon form
    - ... number of pivots
- ERO do not change the row space of the matrix, hence do not change the matrix rank

### Why Row Echelon Form (REF)?
Row echelon form allows us to solve a system of equations $Ux = b$ for $x$ using back substitution.
- Solves for $x$ moving from bottom to top.

$$
x_i = \frac{b_i - \sum_{j=i+1}^{n} U_{i,j} x_j}{U_{i,i}} \quad i = n, n-1, \dots, 1
$$

Forward substitution solves a system of equations $Lx = b$ for $x$.
- Solves for $x$ moving from top to bottom.

$$
x_i = \frac{b_i - \sum_{j=1}^{i-1} L_{i,j} x_j}{L_{i,i}} \quad i = 1, 2, \dots, n
$$

In [2]:
def backsub(U, b):
    """
    backsub solves Ux = b for x using back substitution
    
    :param U: numpy.ndarray  Matrix U, upper triangular
    :param x: numpy.ndarray  Vector b
    :return: numpy.ndarray   Solution x
    """
    assert U.shape[1] == b.shape[0], "U and b not compatible"
    assert np.all(np.diag(U)), "U contains zero pivot"

    m, n = U.shape
    x = np.zeros(n)
    for i in range(m-1, -1, -1):
        x[i] = (b[i] - np.sum(np.dot(U[i,i+1:], x[i+1:]))) / U[i,i]
    return x


n = 4
U = np.triu(np.random.random((n, n)))
x = np.random.random(n)
b = U @ x

# Solve Ux = b for x.
xprime = backsub(U, b)

np.testing.assert_almost_equal(xprime, x)

In [3]:
def fwdsub(L, b):
    """
    fwdsub solves Lx = b for x using forward substitution
    
    :param L: numpy.ndarray  Matrix L, lower triangular
    :param x: numpy.ndarray  Vector b
    :return: numpy.ndarray   Solution x
    """
    assert L.shape[1] == b.shape[0], "U and b not compatible"
    assert np.all(np.diag(L)), "L contains zero pivot"
    
    m, n = L.shape
    x = np.zeros(n)
    for i in range(m):
        x[i] = (b[i] - np.sum(np.dot(L[i,:i], x[:i]))) / L[i,i]
    return x


n = 4
L = np.tril(np.random.random((n, n)))
x = np.random.random(n)
b = L @ x

# Solve Lx = b for x.
xprime = fwdsub(L, b)

np.testing.assert_almost_equal(xprime, x)

## 10.4 Gaussian elimination

### LU Decomposition
LU decomposition solves the linear system of equations $Ax = b$ by factorizing matrix $A$ into lower triangular matrix $L$ and upper triangular matrix $U$.
- Factorization refers to rewriting a mathematical object as the product of other objects

1. Factorize $A$ to product $LU$
2. Rewrite $Ax = b$ as $LUx = b$
3. Solve $Ly = b$ for $y$
4. Solve $Ux = y$ for $x$

Notes
- Technique suitable only for square matrices eg $A$ is a $n \times n$ matrix
- Since factorization is independent of the right hand side, it can be performed once and reused for multiple right hand sides

### Gaussian Elmination
Gaussian Elimination solves the linear system of equations $Ax = b$ using REF.

1. Augment $A$ with $b$ to produce the matrix $[ A | b ]$
2. Use elementary row operations to transform augmented matrix to row echelon form $\text{REF}([ A | b ]) = [U | c]$
3. Use back substitution to solve upper triangular system $Ux = c$ for $x$

In [4]:
def lu(A):
    """
    lu factorizes the square matrix A to [L|U]

    Rather than modify A in place, we return a new matrix containing [L|U].

    L appears in the subdiagonal entries of the returned matrix
    U appears in the diagonal and superdiagonal entries of the returned matrix

    NOTE(mmorais): LU factorization without pivoting is unstable.

    :param A: numpy.ndarray  Matrix A
    :return: numpy.ndarray   Matrix [L|U]
    """
    assert A.shape[0] == A.shape[1], "A is not square"
    assert np.all(np.diag(A)), "A contains zero pivot"

    n = A.shape[0]
    LU = np.copy(A)
    for j in range(n):  # Columns.
        for i in range(j+1, n):  # Rows.
            # Compute the scalar multiple to eliminate current row.
            LU[i,j] = LU[i,j] / LU[j,j]
            # Row_i = Row_i - LU_i,j * Row_j
            LU[i,j+1:] += -1.0 * LU[i,j] * LU[j,j+1:]
    return LU


def asLandU(LU):
    """
    asLandU returns L and U as separate matrices from LU

    :param LU: numpy.ndarray          Matrix LU
    :return: tuple of numpy.ndarray   Matrix L and U
    """
    L = np.tril(LU, k=-1) + np.eye(LU.shape[0])
    U = np.triu(LU)
    return L, U


n = 4
A = np.random.random((n, n))

# Factorize A into [L|U].
LU = lu(A)

# Verify that A = LU.
L, U = asLandU(LU)

np.testing.assert_almost_equal(L @ U, A, err_msg="A != LU")

In [5]:
def solve_lu(A, b):
    """
    solve_lu solves the linear system Ax = b for x using LU factorization

    :param A: numpy.ndarray  Matrix A
    :param x: numpy.ndarray  Vector b
    :return: numpy.ndarray   Solution x
    """
    assert A.shape[0] == A.shape[1], "A is not square"
    assert A.shape[0] == b.shape[0], "A and b not compatible"

    # Factorize A to LU to rewrite Ax = b as LUx = b.
    L, U = asLandU(lu(A))

    # Solve Ly = b for y using forward substitution.
    y = fwdsub(L, b)

    # Solve Ux = y for x using back substitution.
    x = backsub(U, y)

    return x


n = 4
A = np.random.random((n, n))
x = np.random.random(n)
b = A @ x

# Verify solution for x returned by solve_lu matches x.
xprime = solve_lu(A, b)

np.testing.assert_almost_equal(A @ xprime, b, err_msg="Ax != b")
np.testing.assert_almost_equal(xprime, x, err_msg= "x != xprime")

In [6]:
def gaussian_elimination(A):
    """
    gaussian_elimination transforms A to U.

    Rather than modify A in place, we return a new matrix containing U.

    NOTE(mmorais): Gaussian elimination without pivoting is unstable.

    :param A: numpy.ndarray  Matrix A
    :return: numpy.ndarray   Matrix U
    """
    assert np.all(np.diag(A)), "A contains zero pivot"

    m, n = A.shape
    U = np.copy(A)
    for j in range(n):  # Columns.
        for i in range(j+1, m):  # Rows.
            # Compute the scalar multiple to eliminate current row.
            mi = -1.0 * U[i,j] / U[j,j]
            # Row_i = Row_i + mi * Row_j
            U[i,:] += mi * U[j,:]
    return U


def solve_gaussian_elimination(A, b):
    """
    solve_gaussian_elimination solves the linear system Ax = b for x using Gaussian elimination

    :param A: numpy.ndarray  Matrix A
    :param x: numpy.ndarray  Vector b
    :return: numpy.ndarray   Solution x
    """
    assert A.shape[0] == A.shape[1], "A is not square"
    assert A.shape[0] == b.shape[0], "A and b not compatible"

    # Augment A and b to produce a single matrix [A|b].
    Ab = np.hstack((A, b[:, np.newaxis]))

    # Use elementary row operations to transform REF([A|b]) = [U|c].
    Uc = gaussian_elimination(Ab)

    # Solve Uc = y for x using back substitution.
    m, n = A.shape
    x = backsub(Uc[:,:n], Uc[:,n:])

    return x


n = 4
A = np.random.random((n, n))
x = np.random.random(n)
b = A @ x

# Verify solution for x returned by solve_gaussian_elimination matches x.
xprime = solve_gaussian_elimination(A, b)

np.testing.assert_almost_equal(A @ xprime, b, err_msg="Ax != b")
np.testing.assert_almost_equal(xprime, x, err_msg= "x != xprime")

## 10.5 Row-reduced echelon form

### Row-reduced Echelon Form aka RREF
Row-reduced echelon form takes echelon form a little further by 
1. Rescaling nonzero rows by a scalar constant so that pivots have a value of 1
2. Applying row reduction to the remaining rows so that the pivot element is the only nonzero element in each column

Example
$$
\text{RREF}(A) = 
\begin{bmatrix}
1 & 0 & a \\
0 & 1 & b \\
0 & 0 & 0 \\
\end{bmatrix}
$$

Notes
- RREF of a matrix is unique

In [7]:
A = np.array([[1,2,3],[4,5,6],[7,8,9]])

print(A)

# Verify that the number of pivots (2) is equal to the matrix rank.
rrefA, _ = sym.Matrix(A).rref()
print(np.array(rrefA))
np.testing.assert_equal(np.linalg.matrix_rank(A), 2)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 0 -1]
 [0 1 2]
 [0 0 0]]


## 10.6 Gauss-Jordan elimination

### Gauss-Jordan Elimination
Gauss-Jordan Elimination solves the linear system of equations $Ax = b$ using RREF.

1. Augment $A$ with $b$ to produce the matrix $[ A | b ]$
2. Use elementary row operations to transform augmented matrix $\text{RREF}([ A | b ]) = [ I | x ]$

Since Gauss-Jordan uses RREF, using back substitution to solve for $x$ is not required. 


## 10.7 Possibilities for solutions
(_Repeat from 10.1_)

Every solution to the system of equations $Ax = b$ has one of the following outcomes:
1. System has no solutions
    - $A$ is singular
    - $A$ is rank deficient
    - $b$ does not span $A$
2. System has exactly one solution
    - $A$ is nonsingular
    - $A$ is full rank
    - $\text{RREF}(A) = I$
    - $b$ spans $A$ and is linearly independent
    - $N(A) = \emptyset$
3. System has an infinite number of solutions
    - $A$ is singular
    - $A$ is rank deficient
    - $b$ spans $A$ and is linearly dependent

## 10.8 Matrix spaces, row reduction
- ERO does not change rank of the matrix
- ERO does not change the row space of the matrix
- ERO might change the column space of the matrix

## 10.11 Coding challenges

> 1. Create a random coefficient matrix A and variables x. Compute the constants (right hand side) using $Ax = b$.
>
> Verify the following facts about the matrix.
>
> a. Verify the matrix is full rank (nonsingular)
>
> b. Verify the determinant of the matrix is nonzero
>
> c. Solve for x using A and b
>
> d. Find the inverse of the matrix $A^{-1}$ and confirm that $A^{-1} b = x$
>
> e. Use Gauss-Jordan elimination to find x via RREF([A|b]) -> [I|x]

In [8]:
n = 4
A = np.random.random((n, n))
x = np.random.random(n)
b = A @ x

# (a) Verify the matrix is full rank (nonsingular)
np.testing.assert_equal(np.linalg.matrix_rank(A), n)

# (b) Verify the determinant of the matrix is nonzero
np.testing.assert_equal(np.not_equal(np.linalg.det(A), np.zeros(1)), True)

# (c) Solve for x using A and b
np.testing.assert_almost_equal(np.linalg.solve(A, b), x)

# (d) Find the inverse of the matrix $A^{-1}$ and confirm that $A^{-1} b = x$
Ainv = np.linalg.inv(A)
np.testing.assert_almost_equal(Ainv @ b, x)

# (e) Use Gauss-Jordan elimination to find x via RREF([A|b]) -> [I|x]
Ab = np.hstack((A, b[:, np.newaxis]))
rrefAb, _ = sym.Matrix(Ab).rref()
np.testing.assert_almost_equal(np.array(rrefAb[:,-1]), x[:, np.newaxis])

> 2. The margin notes in section 10.5 describe patterns in the RREF of a matrix that depend on the size and shape of the matrix. The idea is that every RREF is essentially the identity matrix, possibly with additional rows of zeros or columns of nonzero numbers. Create random matrices that are (1) square, (2) wide, and (3) tall and compute and comment on the RREF of those matrices.

In [9]:
# Square matrix
# RREF(A) = I
# Matrix is 
m, n = 4, 4
A = np.random.random((m, n))
rrefA, _ = sym.Matrix(A).rref()
print("square:\n", np.array(rrefA))

# Wide matrix
# RREF(A) = | I | X |
m, n = 4, 6
A = np.random.random((m, n))
rrefA, _ = sym.Matrix(A).rref()
print("wide:\n", np.array(rrefA))

# Tall matrix
# RREF(A) = | I |
#           | 0 |
m, n = 6, 4
A = np.random.random((m, n))
rrefA, _ = sym.Matrix(A).rref()
print("tall:\n", np.array(rrefA))

square:
 [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
wide:
 [[1 0 0 0 0.508478057780764 -1.06241044077527]
 [0 1 0 0 0.132370683472273 0.434552481941654]
 [0 0 1 0 0.512552197917528 0.775237860017523]
 [0 0 0 1 0.470264579057402 0.471336867403749]]
tall:
 [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]]
